In [1]:
!nvidia-smi

Sun Aug  8 14:39:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.56       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:37:00.0 Off |                    0 |
| N/A   27C    P8     8W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:86:00.0 Off |                    0 |
| N/A   

In [9]:
import os
import sys

import time
import torch
import torchvision
import torch.profiler
import albumentations as A
from multiprocessing import Manager
from torch.nn.utils import clip_grad_norm_
from albumentations.pytorch import ToTensorV2
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data.dataloader import DataLoader

from data.datasources import build_datasource
from data.datasets.memory_dataset import DatasetCache
from data.samplers import RandomIdentitySampler, RandomIdentitySampler
from data.datasets.reid_dataset import ReidDataset
from losses import build_losses
from trainers.base import BaseTrainer
from callbacks import Tqdm, FreezeLayers
from models import build_model
from optimizers import build_optimizers
from schedulers import build_lr_scheduler
from utils import MetricTracker
from models.activations import get_activations
from utils.ema import ModelEmaV2
from metrics.accuracy import accuracy

In [10]:
import json


def get_config(id, log_folder="saved/logs", info_filename="info.log"):
    with open(os.path.join(log_folder, id, "info.log"), "r") as f:
        line = f.readline()
    return json.loads(
        line[line.find(r"""{'timezone': """) : -1]
        .replace("'", '"')
        .replace("True", "true")
        .replace("False", "false")
        .replace("None", "null")
    )

In [11]:
def get_datasource(config):
    return build_datasource(
        name=config["data"]["name"],
        root_dir=config["data"]["data_dir"],
        #     root_dir="/home/coder/project/datasets",
        download=False,
        extract=False,
        use_tqdm=config["data"]["use_tqdm"],
        relabel=config["data"]["relabel"],
    )

In [12]:
from utils.config import remove_name_config


def get_model(config, num_classes, id, model_name):
    model = build_model(
        name=config["model"]["name"],
        num_classes=num_classes,
        backbone_name=config["model"]["backbone_name"],
        activation=get_activations(
            config["model"]["activation"]["name"],
            **remove_name_config(config["model"]["activation"]),
        ),
        pretrained=config["model"]["pretrained"],
        progress=config["model"]["progress"],
        pooling_name=config["model"]["pooling_name"],
        neck_feat=config["model"]["neck_feat"],
        head_name=config["model"]["head_name"],
        scale=config["model"]["scale"],
        margin=config["model"]["margin"],
        is_inference=True,
    )

    checkpoint = torch.load(
        os.path.join(config["trainer"]["checkpoint_dir"], id, model_name),
        map_location="cpu",
    )

    model.load_state_dict(checkpoint["state_dict"])

    return model

In [13]:
from data.transforms import build_transform


def get_dataloader(config, datasource):
    transform = build_transform(config["data"]["image_size"], is_training=False)

    dataset = dict()
    dataset["query"] = ReidDataset(
        data=datasource.get_data("query"),
        transform=transform,
        transform_lib="torchvision",
    )

    dataset["gallery"] = ReidDataset(
        data=datasource.get_data("gallery"),
        transform=transform,
        transform_lib="torchvision",
    )

    dataloader = dict()

    dataloader["query"] = DataLoader(
        dataset["query"],
        batch_size=128,
        shuffle=False,
        drop_last=False,
    )

    dataloader["gallery"] = DataLoader(
        dataset["gallery"],
        batch_size=128,
        shuffle=False,
        drop_last=False,
    )
    return dataloader

In [14]:
import numpy as np
from tqdm import tqdm


def feature_extractor(model, data_loader, device):
    r"""Extract feature from dataloader
    Args:
        model (models):
        data_loader (Dataloader):
        device (int): torch.device('cpu') if use_gpu == 0 else torch.device(n_gpu)
    Return:
    """
    model.eval()
    feature, label, camera = [], [], []
    with torch.no_grad():
        model.to(device)
        with tqdm(total=len(data_loader)) as pbar:
            for i, batch in enumerate(data_loader):
                x, y, cam_id = batch
                x = x.to(device)
                e = model(x)
                feature.append(e.cpu())
                label.extend(y)
                camera.extend(cam_id)
                pbar.update(1)
    feature = torch.cat(feature, dim=0)
    label = np.asarray(label)
    camera = np.asarray(camera)
    return feature, label, camera

In [15]:
import os
from metrics.distances import cosine_dist
from metrics.mAP import evaluate_rank


def get_mAP(config, id, model, dataloader, device):
    query_feature, query_label, query_camera = feature_extractor(
        model, dataloader["query"], device
    )

    os.makedirs(os.path.join(config["trainer"]["output_dir"], id), exist_ok=True)

    gallery_feature, gallery_label, gallery_camera = feature_extractor(
        model, dataloader["gallery"], device
    )

    if config["loss"]["triplet_distance_mode"] == "euclidean":
        distance = torch.cdist(query_feature, gallery_feature)
    else:
        distance = cosine_dist(query_feature, gallery_feature)

    cmc, all_AP, all_INP = evaluate_rank(
        distance, query_label, gallery_label, query_camera, gallery_camera
    )

    mAP = np.mean(all_AP)
    mINP = np.mean(all_INP)

    rank = dict()
    for r in [1, 5, 10]:
        rank["Rank-{}".format(r)] = cmc[r - 1] * 100

    return mAP, mINP, rank

/home/hiennx/workspace/models/person_reid/rep_reid/metrics/mAP.py:19: UserWarning: Cython rank evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


In [16]:
import logging
from logger import setup_logging


def main(id, device, model_name, before_285=False):
    config = get_config(id)

    os.makedirs(os.path.join(config["trainer"]["output_dir"], id), exist_ok=True)

    setup_logging(os.path.join(config["trainer"]["output_dir"], id))

    logger = logging.getLogger("test")
    logger.info(f"ID: {id}")
    logger.info(f"model_name: {model_name}")

    if before_285:
        config["model"]["backbone_name"] = "resnet50"

    datasource = get_datasource(config)

    model = get_model(config, len(datasource.get_classes()), id, model_name)

    dataloader = get_dataloader(config, datasource)

    mAP, mINP, rank = get_mAP(config, id, model, dataloader, device)

    logger.info(f"mAP: {mAP}")
    logger.info(f"mINP: {mINP}")
    for key, value in rank.items():
        logger.info(f"{key}: {value}")

    logger.info("")

In [17]:
import os

list_id = [x for x in os.listdir("saved/logs") if x.startswith("0807_201910")]
list_id.sort(reverse=True)

list_id

['0807_201910']

In [18]:
!nvidia-smi

Sun Aug  8 14:39:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.56       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:37:00.0 Off |                    0 |
| N/A   27C    P8     8W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:86:00.0 Off |                    0 |
| N/A   

In [19]:
list_model_name = [
    "model_best_triplet_loss.pth",
    "model_last.pth",
    "model_best_id_loss.pth",
    "model_best_loss.pth",
    "model_best_top1.pth",
]

device = torch.device("cuda:0")

In [20]:
for model_name in list_model_name:
    for id in list_id:
        try:
            main(id, device, model_name)
        except Exception as e:
            print(e)

ID: 0807_201910
model_name: model_best_triplet_loss.pth


  0%|          | 0/27 [00:00<?, ?it/s]/opt/conda/envs/reid/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████| 125/125 [01:43<00:00,  1.21it/s]


mAP: 0.8690589616693651
mINP: 0.6048882468719171
Rank-1: 95.1603353023529
Rank-5: 98.54512810707092
Rank-10: 98.99049997329712

ID: 0807_201910
model_name: model_last.pth


100%|██████████| 125/125 [01:43<00:00,  1.21it/s]


mAP: 0.8709392051214483
mINP: 0.6079838044852977
Rank-1: 95.33848166465759
Rank-5: 98.57482314109802
Rank-10: 99.02018904685974

ID: 0807_201910
model_name: model_best_id_loss.pth


...

mAP: 0.8694681463529129
mINP: 0.6089862068522865
Rank-1: 94.98218297958374
Rank-5: 98.45605492591858
Rank-10: 98.87173175811768

ID: 0807_201910
model_name: model_best_loss.pth


100%|██████████| 125/125 [01:44<00:00,  1.20it/s]


mAP: 0.8694681463529129
mINP: 0.6089862068522865
Rank-1: 94.98218297958374
Rank-5: 98.45605492591858
Rank-10: 98.87173175811768

ID: 0807_201910
model_name: model_best_top1.pth


100%|██████████| 125/125 [01:43<00:00,  1.21it/s]


mAP: 0.86893581391267
mINP: 0.6038501630000047
Rank-1: 95.21971344947815
Rank-5: 98.48574995994568
Rank-10: 98.84204268455505



In [21]:
(3**2086) % 440

289